## Tutorial notebook

This Jupyter notebook shows how to perform a basic analysis of γ-ray photon-count maps using the convolutional neural network-based method presented in [arXiv:2107.09070](http://arxiv.org/abs/2107.09070). 

In this example, the photon-count maps consist of **three** different emission components:    
1. *Fermi* bubbles (Poissonian)
2. Galactic Center Excess (point source-like, single population)
3. Isotropic point sources (point source-like, two populations in each map).

As discussed in the paper, for the point source-like templates the Poissonian case is included as the limit of ultra-faint point source emission (<< 1 photon expected per source) where the neural network can no longer distinguish point sources from Poissonian emission.

To consider different scenarios (e.g. other templates, more training data, different network architectures, etc.), simply modify the sample parameter file ```GCE_NN/parameter_files/parameters.py``` accordingly.
The available templates can be viewed in the function ```get_templates()``` in ```GCE/data_utils.py```.

Also, if you don't have access to a GPU and just want to try out the code, it is recommended to reduce the number of training steps in the ```parameters.py```
file in the folder ```parameter_files``` under "Training settings" from ```2500``` to e.g. ```p_train['num_steps'] = 500``` to reduce the
training time. In this case, you will see a warning
```
"WARNING:tensorflow:There are non-GPU devices in `tf.distribute.Strategy`...
```

In [1]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import healpy as hp
import os

In [2]:
%pip list | grep gce-nn   # check if gce-nn module is there
import GCE.gce

First, we need to **initialize** an analysis object.

In [ ]:
gce = GCE.gce.Analysis()

Now, let's **load the parameters** from the parameter file in the parameter_files folder.

In [ ]:
gce.load_params("../parameter_files/parameters_redo.py")

We can take a look at the loaded parameters:

In [ ]:
gce.print_params()

The parameters are stored in gce.p and can also be accessed group-wise. For example, the Poissonian (P) and point-source (PS) templates used in this analysis can be viewed with

In [ ]:
gce.p.mod

and the data-related settings (such as the exposure map, the mask for the region of interest, as well as whether the *Fermi* point-spread function at 2 GeV shall be applied) are stored in

In [ ]:
gce.p.data

Now, let's generate some simulated Monte Carlo photon-count maps for each of the templates. The relevant parameters are stored in the field "tt" (training and testing data) - most importantly the priors, as well as the number of maps given by "n_chunk" (each chunk will be saved in a single file) times the number of simulations per chunk.

In [ ]:
gce.p.tt

To **generate** the template maps, we can simply run

In [ ]:
# Ray settings (for parallelized data generation)
# ray_settings = {"num_cpus": 4, "object_store_memory": 2000000000}
# ray_settings = {"num_cpus": 4}  # select the number of CPUs here
# gce.generate_template_maps(ray_settings, n_example_plots=5, job_id=0)

Some example maps (whose number is determined by ```n_example_plots``` above) for each template can be viewed in the folder ```GCE_NN/data/Template_maps/Example_128```.

The next step is to **combine** (i.e. sum up) the individual template maps to obtain the final training, validation, and testing maps. Internally, this is done in two steps: 1) the filenames of the template maps for each of these three subsets are stored in a file, and 2) the template maps are combined and saved.

In [ ]:
# gce.combine_template_maps(save_filenames=True, do_combine=True)

NOTE: if data has already been generated, the corresponding parameters can be directly loaded from the template maps / combined maps folders, e.g.

```
gce.load_params("../data/Template_maps/Test_128")
gce.load_params("../data/Combined_maps/Test_comb_128")
```


Next, we need to build the **data processing pipeline** that will feed the combined photon-count maps to the neural network.

In [ ]:
gce.build_pipeline()

We can use the method ```get_samples()``` to get photon-count maps and their associated labels from the datasets **train** (used for training), **val** (used as an independent validation dataset during training), and **test** (used for testing once the training is finished) 

In [ ]:
samples = gce.datasets["test"].get_samples(1)
data, labels = samples["data"], samples["label"]  # samples contains data and labels (flux fractions & SCD histograms)
print("Shapes:")
print("  Data", data.shape)  # n_samples x n_pix_in_ROI
print("  Flux fractions", labels[0].shape)  # n_samples x n_templates
print("  SCD histograms", labels[1].shape)  # n_samples x n_bins x n_PS_templates

Let's take a look at a combined map. The maps are compressed and only contain the pixels that lie within the ROI - the method ```decompress()``` returns the full-sky map that can be fed to the healpy functions.

We will plot 
1. the **photon-count map**, 
2. the rescaled version in **'flux' space** as shown to the neural network (divided by exposure correction), and 
3. the *Fermi* **exposure correction**.

In [ ]:
# NOTE: the maps are stored in NEST format
map_to_plot = 0
r = gce.p.data["outer_rad"] + 1
hp.cartview(gce.decompress(data[map_to_plot] * gce.template_dict["rescale_compressed"]), nest=True,
            title="Simulated data: Count space", lonra=[-r, r], latra=[-r, r])
hp.cartview(gce.decompress(data[map_to_plot]), nest=True,
            title="Simulated data: Flux space", lonra=[-r, r], latra=[-r, r])
hp.cartview(gce.decompress(gce.template_dict["rescale_compressed"], fill_value=np.nan), nest=True,
            title="Fermi exposure correction", lonra=[-r, r], latra=[-r, r])
plt.show()

Let's also plot the real *Fermi* map in our region of interest. Of course, it looks quite different from our simulated maps because we only included the *Fermi* bubbles, the GCE, and isotropic point sources in this example (so we are completely ignoring the diffuse Galactic foregrounds, which are responsible for the majority of photon counts).

In [ ]:
fermi_counts = gce.datasets["test"].get_fermi_counts()
hp.cartview(gce.decompress(fermi_counts * gce.generators["test"].settings_dict["rescale_compressed"]), nest=True,
            title="Fermi data: Count space", max=100, lonra=[-r, r], latra=[-r, r])
# hp.cartview(gce.decompress(fermi_counts), nest=True, title="Fermi data: Flux space", max=100)
plt.show()

Now, it's time to **build** our neural network:


In [ ]:
gce.build_nn()

*NOTE*: Once the neural network has been trained, **loading** is as easy as ```gce.load_nn()```.

In [ ]:
# gce.load_nn()

Let's **train** our neural network to predict 
1. the **flux fractions** of the different templates (using a negative maximum log-likelihood loss function), and 
2. the **SCD histograms** of the GCE and isotropic point source populations (using the *Earth Mover's pinball loss*, see [arXiv:2106.02051](https://arxiv.org/abs/2106.02051)).

In [ ]:
# gce.train_nn("flux_fractions")

In [ ]:
gce.train_nn("histograms")

Finally, let's **evaluate** the performance of our neural network on simulated test data.

In [ ]:
n_samples = 20
test_samples = gce.datasets["test"].get_samples(n_samples)
test_data, test_ffs, test_hists = test_samples["data"], test_samples["label"][0], test_samples["label"][1]
tau = np.arange(5, 100, 5) * 0.01  # quantile levels for SCD histograms, from 5% to 95% in steps of 5%
pred = gce.predict(test_data, tau=tau, multiple_taus=True)  # get the NN predictions

In [ ]:
# Make some plots (will be saved in the models folder)
gce.plot_nn_architecture()
gce.plot_flux_fractions(test_ffs, pred)
gce.plot_histograms(test_hists, pred, plot_inds=np.arange(9))
gce.plot_maps(test_data, decompress=True, plot_inds=np.arange(9))
plt.show()

Clearly, the training dataset is too small and the training was too short to obtain accurate and precise predictions. Still, the neural networks have already learned *something*, and the predictions are roughly in the right ballpark.